In [1]:
import numpy as np
import cv2
from PIL import Image
from sklearn.decomposition import PCA
from scipy.fftpack import dct
import imagehash
import warnings

In [2]:
# Utility functions for video processing

def open_video(video_path):
    return cv2.VideoCapture(video_path)

def save_video(output_path, fps, frame_size, is_color=True):
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    return cv2.VideoWriter(output_path, fourcc, fps, frame_size, is_color)

def process_and_save_video(video_path, output_path, process_func, is_color=True):
    cap = open_video(video_path)
    out = save_video(output_path, cap.get(cv2.CAP_PROP_FPS),
                     (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))), is_color)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = process_func(frame)
        out.write(processed_frame)

    cap.release()
    out.release()

In [3]:
def thph_hash(video_path, hash_size=8):
    cap = open_video(video_path)
    previous_frame_hash = None
    video_hashes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to grayscale and resize
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, (64, 64), interpolation=cv2.INTER_LANCZOS4)
        pixels = resized_frame.astype(float)

        # Apply PCA
        pca = PCA(n_components=hash_size)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            pca_transformed = pca.fit_transform(pixels)

        if np.isnan(pca_transformed).any():
            pca_transformed = np.nan_to_num(pca_transformed)

        # Apply DCT
        dct_transformed = dct(dct(pca_transformed, axis=0), axis=1)
        dct_low_freq = dct_transformed[:hash_size, :hash_size]
        dct_median_val = np.median(dct_low_freq)
        dct_hash_array = (dct_low_freq > dct_median_val).astype(int)
        dct_hash_hex = ''.join(str(x) for x in dct_hash_array.flatten())

        # Apply difference hashing for temporal changes
        pil_image = Image.fromarray(resized_frame)
        current_frame_hash = imagehash.dhash(pil_image)

        if previous_frame_hash is not None:
            diff_hash_array = np.array(current_frame_hash.hash) ^ np.array(previous_frame_hash.hash)
            temporal_diff_hash = diff_hash_array.flatten().astype(int)
        else:
            temporal_diff_hash = np.zeros((hash_size * hash_size,), dtype=int)

        previous_frame_hash = current_frame_hash

        # Combine PCA-DCT and Temporal dHash results
        combined_hash_array = np.concatenate([dct_hash_array.flatten(), temporal_diff_hash])
        combined_hash_string = ''.join(str(x) for x in combined_hash_array)
        combined_hash_hex = hex(int(combined_hash_string, 2))[2:].zfill(len(combined_hash_array) // 4)

        video_hashes.append(combined_hash_hex)

    cap.release()
    return video_hashes

In [4]:
def hex_to_bin(hex_str):
    return bin(int(hex_str, 16))[2:].zfill(256)

def hamming_distance(bin_str1, bin_str2):
    return sum(c1 != c2 for c1, c2 in zip(bin_str1, bin_str2))

def similarity_score(original_hashes, edited_hashes):
    scores = []
    for original, edited in zip(original_hashes, edited_hashes):
        bin_original = hex_to_bin(original)
        bin_edited = hex_to_bin(edited)
        distance = hamming_distance(bin_original, bin_edited)
        score = 1 - (distance / len(bin_original))
        scores.append(score)
    return scores

def compute_similarity_scores(original_hashes, modified_hashes):
    if not original_hashes or not modified_hashes:
        print("Error: One or both sets of hashes are empty.")
        return [], 0.0

    if len(original_hashes) != len(modified_hashes):
        print("Warning: The number of hashes in the original and modified videos do not match.")
        return [], 0.0

    scores = similarity_score(original_hashes, modified_hashes)
    average_score = sum(scores) / len(scores) if scores else 0.0
    return scores, average_score

In [5]:
# Specific video processing functions (Filter Operations)

def apply_gaussian_blur(frame, kernel_size=(5, 5)):
    return cv2.GaussianBlur(frame, kernel_size, 0)

def apply_sharpening(frame):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(frame, -1, kernel)

def apply_smoothing(frame, diameter=15, sigma_color=75, sigma_space=75):
    return cv2.bilateralFilter(frame, diameter, sigma_color, sigma_space)

def apply_edge_detection(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_frame, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

def apply_color_filtering(frame, lower_bound=(35, 50, 50), upper_bound=(85, 255, 255)):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_frame, lower_bound, upper_bound)
    return cv2.bitwise_and(frame, frame, mask=mask)

In [6]:
# Specific video processing functions (Transformation Operations)

def apply_compression(frame, quality=30):
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    result, encimg = cv2.imencode('.jpg', frame, encode_param)
    return cv2.imdecode(encimg, 1)

def apply_rotation(video_path, output_path, angle=90):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Adjust frame dimensions for 90° or 270° rotation
    if angle in [90, 270]:
        new_width, new_height = height, width
    else:
        new_width, new_height = width, height

    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (new_width, new_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if angle == 90:
            rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        elif angle == 180:
            rotated_frame = cv2.rotate(frame, cv2.ROTATE_180)
        elif angle == 270:
            rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        else:
            rotated_frame = frame

        out.write(rotated_frame)

    cap.release()
    out.release()

def apply_scaling(video_path, output_path, new_size=(320, 240)):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), new_size)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, new_size)
        out.write(resized_frame)

    cap.release()
    out.release()

def apply_cropping(video_path, output_path, crop_rect=(50, 50, 200, 200)):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    crop_width = crop_rect[2]
    crop_height = crop_rect[3]

    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (crop_width, crop_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        cropped_frame = frame[crop_rect[1]:crop_rect[1] + crop_rect[3], crop_rect[0]:crop_rect[0] + crop_rect[2]]
        out.write(cropped_frame)

    cap.release()
    out.release()
    
def apply_flipping(video_path, output_path, flip_code):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        flipped_frame = cv2.flip(frame, flip_code)
        out.write(flipped_frame)

    cap.release()
    out.release()

# Apply Flipping to the video
apply_flipping('input_video.mp4', 'flipped_video_horizontal.mp4', 1)  # Horizontal flip
apply_flipping('input_video.mp4', 'flipped_video_vertical.mp4', 0)    # Vertical flip

def apply_perspective_transformation(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS),
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the transformation matrix
    pts1 = np.float32([[0, 0], [width - 1, 0], [0, height - 1], [width - 1, height - 1]])
    pts2 = np.float32([[0, 0], [width * 0.8, height * 0.2], [width * 0.2, height * 0.8], [width, height]])

    matrix = cv2.getPerspectiveTransform(pts1, pts2)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        transformed_frame = cv2.warpPerspective(frame, matrix, (width, height))
        out.write(transformed_frame)

    cap.release()
    out.release()

In [7]:
# Specific video processing functions (Temporal Operations)

def loop_video(input_path, output_path, loop_count=3):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()

    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for _ in range(loop_count):
        for frame in frames:
            out.write(frame)

    out.release()

def apply_frame_dropping(input_path, output_path, drop_rate):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS) / drop_rate
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % drop_rate == 0:
            out.write(frame)
        frame_count += 1

    cap.release()
    out.release()

def apply_frame_rate_conversion(input_path, output_path, new_fps):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, new_fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()

def apply_speed_manipulation(input_path, output_path, speed_factor):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS) * speed_factor
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()

In [8]:
# Specific video processing functions (Spatial Operations)

def apply_translation(video_path, output_path, tx=30, ty=30):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (width, height))

    # Define translation matrix
    M = np.float32([[1, 0, tx], [0, 1, ty]])

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Apply the translation
        translated_frame = cv2.warpAffine(frame, M, (width, height))
        out.write(translated_frame)

    cap.release()
    out.release()

def apply_zoom(video_path, output_path, zoom_factor=1.5):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate center crop dimensions
        center_x, center_y = width // 2, height // 2
        crop_width, crop_height = int(width / zoom_factor), int(height / zoom_factor)
        crop_x1 = max(0, center_x - crop_width // 2)
        crop_x2 = min(width, center_x + crop_width // 2)
        crop_y1 = max(0, center_y - crop_height // 2)
        crop_y2 = min(height, center_y + crop_height // 2)

        # Crop and resize to original size
        cropped_frame = frame[crop_y1:crop_y2, crop_x1:crop_x2]
        zoomed_frame = cv2.resize(cropped_frame, (width, height))

        out.write(zoomed_frame)

    cap.release()
    out.release()

def apply_affine_transformation(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (width, height))

    # Define affine transformation matrix
    pts1 = np.float32([[50, 50], [200, 50], [50, 200]])
    pts2 = np.float32([[10, 100], [200, 50], [100, 250]])
    M = cv2.getAffineTransform(pts1, pts2)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Apply the affine transformation
        affine_transformed_frame = cv2.warpAffine(frame, M, (width, height))
        out.write(affine_transformed_frame)

    cap.release()
    out.release()

# Example usage
video_path = r"C:\Users\prias\Downloads\THPH\CGI Animated Short Film_ _Watermelon A Cautionary Tale_ by Kefei Li & Connie Qin He _ CGMeetup.mp4"

In [9]:
# Apply video transformations
apply_rotation(video_path, 'rotated_video_90.mp4', angle=90)
apply_scaling(video_path, 'resized_video.mp4', new_size=(320, 240))
apply_cropping(video_path, 'cropped_video.mp4', crop_rect=(50, 50, 200, 200))
loop_video(video_path, 'looped_video.mp4', loop_count=3)
apply_translation(video_path, 'translated_video.mp4', tx=30, ty=30)
apply_zoom(video_path, 'zoomed_video.mp4', zoom_factor=1.5)
apply_affine_transformation(video_path, 'affine_transformed_video.mp4')

In [10]:
# Apply additional video processing
apply_frame_dropping(video_path, 'frame_dropped_video.mp4', drop_rate=2)
apply_frame_rate_conversion(video_path, 'frame_rate_converted_video.mp4', new_fps=15)
apply_speed_manipulation(video_path, 'slowed_down_video.mp4', speed_factor=0.5)

In [13]:
# Process and save videos with different transformations
process_and_save_video(video_path, 'blurred_video.mp4', apply_gaussian_blur)
process_and_save_video(video_path, 'sharpened_video.mp4', apply_sharpening)
process_and_save_video(video_path, 'smoothed_video.mp4', apply_smoothing)
process_and_save_video(video_path, 'edges_video.mp4', apply_edge_detection)
process_and_save_video(video_path, 'color_filtered_video.mp4', apply_color_filtering)
process_and_save_video(video_path, 'compressed_video.mp4', lambda frame: apply_compression(frame, quality=30))
apply_flipping(video_path, 'flipped_video_horizontal.mp4',1)
apply_flipping(video_path, 'flipped_video_vertical.mp4', 0)
apply_perspective_transformation(video_path, 'perspective_transformed_video.mp4')

In [14]:
# Compute the THPH hashes for original and modified videos
original_hashes = thph_hash(video_path)
modified_videos = {
    'Blurred': 'blurred_video.mp4',
    'Sharpened': 'sharpened_video.mp4',
    'Smoothed': 'smoothed_video.mp4',
    'Edges': 'edges_video.mp4',
    'Color Filtered': 'color_filtered_video.mp4',
    'Compressed': 'compressed_video.mp4',
    'Rotated': 'rotated_video_90.mp4',
    'Resized': 'resized_video.mp4',
    'Cropped': 'cropped_video.mp4',
    'Flipped Horizontal': 'flipped_video_horizontal.mp4',
    'Flipped Vertical': 'flipped_video_vertical.mp4',
    'Perspective Transformed': 'perspective_transformed_video.mp4',
    'Translated': 'translated_video.mp4',
    'Zoomed': 'zoomed_video.mp4',
    'Affine Transformed': 'affine_transformed_video.mp4',
    'Frame Dropped': 'frame_dropped_video.mp4',
    'Frame Rate Converted': 'frame_rate_converted_video.mp4',
    'Slowed Down': 'slowed_down_video.mp4',
    'Looped': 'looped_video.mp4'
}

In [15]:
# Compute similarity scores
for transformation, path in modified_videos.items():
    modified_hashes = thph_hash(path)
    print(f"Comparing original and {transformation} hashes:")
    print(f"Number of Original Hashes: {len(original_hashes)}")
    print(f"Number of Modified Hashes: {len(modified_hashes)}")
    scores, avg_score = compute_similarity_scores(original_hashes, modified_hashes)
    if scores:
        print(f"{transformation} Similarity Scores:", scores)
        print(f"Average {transformation} Similarity:", avg_score)
    else:
        print(f"Could not compute similarity scores for {transformation}.")


Comparing original and Blurred hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 4225
Blurred Similarity Scores: [1.0, 0.9609375, 0.9765625, 0.984375, 0.98046875, 0.984375, 0.98046875, 0.984375, 0.984375, 0.9765625, 0.98046875, 0.984375, 0.984375, 0.984375, 0.984375, 0.984375, 0.984375, 0.9765625, 0.98046875, 0.98046875, 0.984375, 0.984375, 0.984375, 0.984375, 0.984375, 0.984375, 0.96484375, 0.97265625, 0.96875, 0.96484375, 0.98046875, 0.984375, 0.96875, 0.96484375, 0.98046875, 0.984375, 0.984375, 0.984375, 0.98046875, 0.98046875, 0.9765625, 0.984375, 0.984375, 0.89453125, 0.9765625, 0.9921875, 0.96875, 0.984375, 0.96875, 0.96875, 0.984375, 1.0, 0.9765625, 0.9921875, 0.984375, 0.9453125, 0.9921875, 0.9921875, 0.9921875, 0.9765625, 0.96875, 0.984375, 0.9140625, 0.953125, 0.9765625, 0.9296875, 0.95703125, 0.9765625, 0.96875, 0.96875, 0.9765625, 0.96875, 0.96875, 0.9765625, 0.96875, 0.96875, 0.9765625, 0.96875, 0.96875, 0.96875, 0.9765625, 0.9765625, 0.96875, 0.96875, 0.9

Comparing original and Sharpened hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 4225
Sharpened Similarity Scores: [1.0, 0.953125, 0.97265625, 0.98828125, 0.9921875, 0.9921875, 0.98828125, 0.984375, 1.0, 0.984375, 0.98828125, 0.9921875, 0.984375, 0.9921875, 1.0, 0.9921875, 0.9921875, 0.96875, 0.984375, 0.9921875, 0.9921875, 1.0, 0.9921875, 0.98828125, 0.98828125, 0.9921875, 0.98046875, 0.98828125, 0.97265625, 0.9765625, 0.98828125, 0.9921875, 0.984375, 0.9921875, 0.9921875, 0.984375, 0.984375, 0.9921875, 0.98828125, 0.98046875, 0.984375, 0.9921875, 0.9921875, 0.90625, 0.9609375, 1.0, 0.9375, 0.9921875, 0.9609375, 0.94921875, 0.98046875, 0.984375, 0.9765625, 1.0, 0.9765625, 0.921875, 0.98828125, 0.94921875, 0.97265625, 0.96875, 0.89453125, 0.921875, 0.90234375, 0.9296875, 0.859375, 0.8671875, 0.8515625, 0.859375, 0.9296875, 0.9375, 0.9375, 0.859375, 0.859375, 0.859375, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 0.875, 0.8671875, 0.875, 0.8671875

Comparing original and Smoothed hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 4225
Smoothed Similarity Scores: [1.0, 0.9921875, 0.984375, 0.984375, 0.984375, 0.984375, 0.98046875, 0.98046875, 0.984375, 0.96875, 0.98046875, 0.984375, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9921875, 1.0, 1.0, 1.0, 1.0, 1.0, 0.984375, 1.0, 1.0, 0.98828125, 0.98828125, 0.984375, 0.984375, 1.0, 1.0, 0.984375, 0.9921875, 1.0, 1.0, 1.0, 1.0, 0.99609375, 0.9921875, 1.0, 1.0, 1.0, 0.953125, 0.9375, 0.9921875, 0.98828125, 0.98046875, 0.96875, 0.96484375, 0.98046875, 0.9765625, 0.96484375, 0.99609375, 0.9765625, 0.93359375, 0.98828125, 0.9765625, 0.953125, 0.9765625, 0.9609375, 0.9140625, 0.9140625, 0.9609375, 0.9765625, 0.96484375, 0.94921875, 1.0, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.94921875, 0.94921875, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.9921875, 0.992

Comparing original and Edges hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 4225
Edges Similarity Scores: [1.0, 0.921875, 0.94140625, 0.953125, 0.953125, 0.953125, 0.94921875, 0.953125, 0.9453125, 0.9453125, 0.94921875, 0.953125, 0.953125, 0.953125, 0.953125, 0.953125, 0.94921875, 0.94921875, 0.9453125, 0.9453125, 0.94140625, 0.94921875, 0.953125, 0.93359375, 0.94921875, 0.953125, 0.9296875, 0.9375, 0.9375, 0.92578125, 0.94921875, 0.953125, 0.93359375, 0.92578125, 0.953125, 0.953125, 0.953125, 0.953125, 0.94921875, 0.94921875, 0.9296875, 0.94140625, 0.94921875, 0.796875, 0.84375, 0.8359375, 0.8515625, 0.890625, 0.875, 0.84375, 0.82421875, 0.8671875, 0.875, 0.87109375, 0.875, 0.8515625, 0.828125, 0.86328125, 0.8359375, 0.84375, 0.8203125, 0.9140625, 0.87109375, 0.86328125, 0.921875, 0.94140625, 0.94921875, 0.890625, 0.8828125, 0.9453125, 0.9453125, 0.9453125, 0.953125, 0.953125, 0.890625, 0.8984375, 0.8984375, 0.8984375, 0.890625, 0.94140625, 0.89453125, 0.890625, 0.

Comparing original and Color Filtered hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 4225
Color Filtered Similarity Scores: [1.0, 0.859375, 0.87109375, 0.875, 0.875, 0.875, 0.87109375, 0.875, 0.875, 0.87109375, 0.8671875, 0.875, 0.875, 0.875, 0.875, 0.875, 0.875, 0.875, 0.875, 0.875, 0.875, 0.875, 0.875, 0.87109375, 0.87109375, 0.875, 0.859375, 0.859375, 0.859375, 0.85546875, 0.87109375, 0.875, 0.8671875, 0.86328125, 0.87109375, 0.875, 0.875, 0.875, 0.87109375, 0.875, 0.87109375, 0.875, 0.875, 0.7578125, 0.84375, 0.921875, 0.8984375, 0.8828125, 0.90234375, 0.85546875, 0.83203125, 0.8359375, 0.89453125, 0.82421875, 0.890625, 0.90234375, 0.828125, 0.875, 0.875, 0.87890625, 0.87890625, 0.83203125, 0.8515625, 0.83984375, 0.83984375, 0.828125, 0.84765625, 0.8359375, 0.8515625, 0.8515625, 0.82421875, 0.84375, 0.8359375, 0.828125, 0.8359375, 0.828125, 0.828125, 0.828125, 0.8359375, 0.8359375, 0.83203125, 0.84375, 0.83203125, 0.84375, 0.84375, 0.84375, 0.8515625, 0.8515625,

...

Comparing original and Zoomed hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 4225
Zoomed Similarity Scores: [1.0, 0.953125, 0.96875, 0.97265625, 0.97265625, 0.97265625, 0.97265625, 0.9765625, 0.9765625, 0.95703125, 0.9609375, 0.9765625, 0.9765625, 0.9765625, 0.9765625, 0.9765625, 0.9765625, 0.96875, 0.9765625, 0.9765625, 0.9765625, 0.9765625, 0.9765625, 0.97265625, 0.97265625, 0.9765625, 0.953125, 0.9609375, 0.95703125, 0.9375, 0.96484375, 0.9765625, 0.96875, 0.953125, 0.96875, 0.96875, 0.96875, 0.9765625, 0.97265625, 0.97265625, 0.96875, 0.97265625, 0.96875, 0.80859375, 0.86328125, 0.8515625, 0.85546875, 0.796875, 0.87890625, 0.85546875, 0.8203125, 0.8984375, 0.88671875, 0.875, 0.88671875, 0.8828125, 0.88671875, 0.890625, 0.9140625, 0.86328125, 0.87890625, 0.84375, 0.8515625, 0.84765625, 0.86328125, 0.87109375, 0.8671875, 0.8671875, 0.86328125, 0.8671875, 0.8671875, 0.86328125, 0.8671875, 0.86328125, 0.86328125, 0.8671875, 0.86328125, 0.86328125, 0.8671875, 0.875, 

Comparing original and Affine Transformed hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 4225
Affine Transformed Similarity Scores: [1.0, 0.84765625, 0.9140625, 0.8984375, 0.8984375, 0.890625, 0.890625, 0.8828125, 0.890625, 0.8671875, 0.88671875, 0.8984375, 0.88671875, 0.88671875, 0.8984375, 0.8984375, 0.8984375, 0.890625, 0.8984375, 0.8984375, 0.8984375, 0.89453125, 0.89453125, 0.88671875, 0.89453125, 0.8984375, 0.87109375, 0.87890625, 0.86328125, 0.87890625, 0.890625, 0.8984375, 0.87890625, 0.875, 0.89453125, 0.8984375, 0.8984375, 0.8984375, 0.88671875, 0.87890625, 0.88671875, 0.890625, 0.890625, 0.77734375, 0.8203125, 0.85546875, 0.84375, 0.8671875, 0.84765625, 0.875, 0.87890625, 0.8671875, 0.875, 0.86328125, 0.86328125, 0.8671875, 0.86328125, 0.8671875, 0.87890625, 0.8671875, 0.88671875, 0.8984375, 0.8828125, 0.8671875, 0.8671875, 0.83984375, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 0.8671875, 

Comparing original and Frame Dropped hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 2113
Could not compute similarity scores for Frame Dropped.
Comparing original and Frame Rate Converted hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 4225
Frame Rate Converted Similarity Scores: [1.0, 0.9765625, 0.984375, 0.984375, 1.0, 1.0, 0.99609375, 1.0, 1.0, 0.9921875, 1.0, 0.99609375, 1.0, 1.0, 1.0, 1.0, 1.0, 0.98828125, 0.99609375, 1.0, 1.0, 1.0, 1.0, 0.9921875, 0.99609375, 1.0, 0.98828125, 0.98828125, 0.9765625, 0.984375, 1.0, 1.0, 0.98046875, 0.98828125, 1.0, 1.0, 1.0, 1.0, 0.99609375, 0.9921875, 1.0, 1.0, 1.0, 0.9375, 0.984375, 0.9921875, 0.98828125, 0.99609375, 0.9609375, 0.96484375, 0.93359375, 0.9765625, 0.9765625, 0.9921875, 0.98046875, 0.97265625, 0.953125, 0.9765625, 1.0, 0.9921875, 0.9921875, 1.0, 0.9765625, 0.9609375, 0.9765625, 0.97265625, 0.98828125, 0.96875, 0.9921875, 0.9921875, 0.9921875, 1.0, 0.98828125, 0.9921875, 0.9921875, 0.9921875, 0.

Comparing original and Slowed Down hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 4225
Slowed Down Similarity Scores: [1.0, 0.9765625, 0.984375, 0.984375, 1.0, 1.0, 0.99609375, 1.0, 1.0, 0.9921875, 1.0, 0.99609375, 1.0, 1.0, 1.0, 1.0, 1.0, 0.98828125, 0.99609375, 1.0, 1.0, 1.0, 1.0, 0.9921875, 0.99609375, 1.0, 0.98828125, 0.98828125, 0.9765625, 0.984375, 1.0, 1.0, 0.98046875, 0.98828125, 1.0, 1.0, 1.0, 1.0, 0.99609375, 0.9921875, 1.0, 1.0, 1.0, 0.9375, 0.984375, 0.9921875, 0.98828125, 0.99609375, 0.9609375, 0.96484375, 0.93359375, 0.9765625, 0.9765625, 0.9921875, 0.98046875, 0.97265625, 0.953125, 0.9765625, 1.0, 0.9921875, 0.9921875, 1.0, 0.9765625, 0.9609375, 0.9765625, 0.97265625, 0.98828125, 0.96875, 0.9921875, 0.9921875, 0.9921875, 1.0, 0.98828125, 0.9921875, 0.9921875, 0.9921875, 0.9921875, 0.9921875, 1.0, 1.0, 0.9921875, 0.9921875, 0.9921875, 1.0, 0.9921875, 0.9921875, 0.9921875, 0.9921875, 0.9921875, 0.9921875, 0.9921875, 0.9921875, 1.0, 0.9921875, 0.9921875,

Comparing original and Looped hashes:
Number of Original Hashes: 4225
Number of Modified Hashes: 12675
Could not compute similarity scores for Looped.
